In [1]:
import sqlalchemy
import sqlite3
from sqlalchemy import Column, ForeignKey, Integer, String, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import os

In [2]:
# DBSession = sessionmaker(bind=engine)
# session = DBSession()
# field = Field(FieldName="Objective")
# session.add(field)
# session.query(Field).first().id

In [3]:
os.remove("db.db")
engine = sqlalchemy.create_engine("sqlite:///db.db")
Base = declarative_base(bind=engine)
connection = engine.connect()
DBSession = sessionmaker(bind=engine)

In [4]:
class StringText(Base):
    __tablename__ = "stringtexts"
    id = Column(Integer(), nullable=False, primary_key=True)
    stringtext = Column(String(), nullable=False)

    fieldstrings = relationship('FieldString')


class Field(Base):
    __tablename__ = "fields"
    id = Column(Integer(), nullable=False, primary_key=True)
    fieldname = Column(String(), nullable=False)

    fieldstrings = relationship('FieldString')


class FieldString(Base):
    __tablename__ = "fieldstrings"
    id = Column(Integer(), nullable=False, primary_key=True)
    fields_id = Column(Integer(), ForeignKey('fields.id'), nullable=False)
    stringtexts_id = Column(Integer(), ForeignKey('stringtexts.id'), nullable=False)

    sectionfields = relationship('SectionField')


class Model(Base):
    __tablename__ = "models"
    id = Column(Integer(), nullable=False, primary_key=True)
    partnumber = Column(String(), nullable=False)

    modeldescriptor = relationship('ModelDescriptor')


class ModelDescriptor(Base):
    __tablename__ = "modeldescriptor"
    id = Column(Integer(), nullable=False, primary_key=True)
    descriptors_id = Column(Integer(), ForeignKey('descriptors.id'), nullable=False)
    models_id = Column(Integer(), ForeignKey('models.partnumber'), nullable=False)


class Descriptor(Base):
    __tablename__ = "descriptors"
    id = Column(Integer(), nullable=False, primary_key=True)
    descriptor = Column(String(), nullable=False)

    sectiondescriptor = relationship('SectionDescriptor')
    modeldescriptor = relationship('ModelDescriptor')


class SectionVersion(Base):
    __tablename__ = "sectionversion"
    id = Column(Integer(), nullable=False, primary_key=True)
    section_id = Column(Integer(), ForeignKey('section.id'), nullable=False)

    sectionsteps = relationship('SectionStep')
    sectionfields = relationship('SectionField')
    sectiondescriptor = relationship('SectionDescriptor')


class SectionDescriptor(Base):
    __tablename__ = "sectiondescriptor"
    id = Column(Integer(), nullable=False, primary_key=True)
    descriptors_id = Column(Integer(), ForeignKey('descriptors.id'), nullable=False)
    sectionversion_id = Column(Integer(), ForeignKey('sectionversion.id'), nullable=False)


class SectionField(Base):
    __tablename__ = "sectionfields"
    id = Column(Integer(), nullable=False, primary_key=True)
    sectionversion_id = Column(Integer(), ForeignKey('sectionversion.id'), nullable=False)
    fieldstrings_id = Column(Integer(), ForeignKey('fieldstrings.id'), nullable=False)


class SectionStep(Base):
    __tablename__ = "sectionsteps"
    id = Column(Integer(), nullable=False, primary_key=True)
    sectionversion_id = Column(Integer(), ForeignKey('sectionversion.id'), nullable=False)
    steps_id = Column(Integer(), ForeignKey('steps.id'), nullable=False)


class Step(Base):
    __tablename__ = "steps"
    id = Column(Integer(), nullable=False, primary_key=True)
    steptext = Column(String(), nullable=False)
    steptypes_id = Column(Integer(), ForeignKey('steptypes.id'), nullable=False)

    sectionsteps = relationship('SectionStep')


class StepType(Base):
    __tablename__ = "steptypes"
    id = Column(Integer(), nullable=False, primary_key=True)
    name = Column(String(), nullable=False)
    description = Column(String(), nullable=False)

    steps = relationship('Step')


class Section(Base):
    __tablename__ = "section"
    id = Column(Integer(), nullable=False, primary_key=True)
    title = Column(String(), nullable=False)

    sectionversion = relationship('SectionVersion')



In [5]:
Base.metadata.create_all()

In [6]:
session = DBSession()

In [7]:
fields = "Objective", "Prerequisites","Acceptance Criteria", "Equipment", "Topics", "Tags", "Software"
for f in fields:
    field = Field(fieldname=f)
    session.add(field)
session.commit()

In [8]:
sess = DBSession()
section = Section(title="Time Server Configuration")
sess.add(section)
sess.flush()

version = SectionVersion(section_id=section.id)
sess.add(version)
sess.flush()

text = StringText(stringtext="Sync Windows PC to time server")
sess.add(text)
sess.flush()

textfield = FieldString(fields_id=sess.query(Field).filter_by(fieldname="Objective").first().id,
                       stringtexts_id=text.id)
sess.add(textfield)
sess.flush()

objective = SectionField(sectionversion_id=version.id,
                        fieldstrings_id=textfield.id)
sess.add(objective)
sess.commit()